# SEIRX model runs for only TTI

In [6]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [7]:
# paths for I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [8]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''    

    # extract the simulation parameters from the parameter list
    index_case, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, index_case=index_case)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case)

    return row

## Screening parameters

In [9]:
# specifies, whether the index case will be introduced via an employee or a 
# resident
index_cases = ['employee', 'resident']

# ensemble_size
N_runs = 5000

screening_params = [(i, N_runs) for i in index_cases ]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 2 parameter combinations


## B.1.1.7

### Simulation parameters and measures in place

In [10]:
with open('params/TTI_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/TTI_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [11]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/TTI/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_TTI_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [08:59<00:00, 269.53s/it]

CPU times: user 125 ms, sys: 105 ms, total: 230 ms
Wall time: 8min 59s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,17.123992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,17.419063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Wild type

In [12]:
ensmbl_dst = '../data/simulation_results/ensembles/TTI/wild_type'

### Simulation parameters and measures in place

In [13]:
with open('params/TTI_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/TTI_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [14]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_TTI_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [06:11<00:00, 185.58s/it]


CPU times: user 125 ms, sys: 89.7 ms, total: 215 ms
Wall time: 6min 11s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,10.190287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,10.264416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
